In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from airfoil.airfoil import Airfoil, Hole, WingSegment, Decomposer, Hinge
from airfoil.spitfire import SpitfireWing
from numpy.typing import ArrayLike
from itertools import pairwise

import pyvista as pv

In [ ]:
wing = SpitfireWing(half_span=500)
section_positions = np.array([
    0,
    100,
    200,
    300,
    400,
    450,
])
airfoils = wing.create_airfoils(section_positions)

wing.plot(section_positions=section_positions)

In [ ]:
chunks_a, chunks_b = [*pairwise(airfoils)], [*reversed([*pairwise(reversed(airfoils))])]

def add_holes(afs:tuple[Airfoil,Airfoil])->tuple[Airfoil,Airfoil]:
    holes = [
        Hole(diameter_mm=5, position=np.array([ -40,8])),
        Hole(diameter_mm=5, position=np.array([  10,8])),
    ]
    a, b = afs
    a = a.with_holes(holes)
    b = b.with_holes(holes)
    return a, b

def add_airleron(
        afs:tuple[Airfoil,Airfoil],
        xp1:float,
        xp2:float,
        upper_thickness:float=2,
    )->tuple[Airfoil,Airfoil]:
    a, b = afs
    a = a.with_hinge(
        Hinge(position=[xp1, 0]),
        upper_thickness=upper_thickness
    )
    b = b.with_hinge(
        Hinge(position=[xp2, 0]),
        upper_thickness=upper_thickness
    )
    return a, b


chunks_a[0] = add_holes(chunks_a[0])
chunks_b[0] = add_holes(chunks_b[0])
chunks_a[-1] = add_airleron(chunks_a[-1], wing.flap_line(400),wing.flap_line(450))
chunks_a[-2] = add_airleron(chunks_a[-2], wing.flap_line(300),wing.flap_line(400))
chunks_b[-1] = add_airleron(chunks_b[-1], wing.flap_line(450),wing.flap_line(400))
chunks_b[-2] = add_airleron(chunks_b[-2], wing.flap_line(400),wing.flap_line(300))
airfoil_pairs_a:list[WingSegment] = [
    WingSegment(a, b, length=length)
    for (a, b), length
    in zip(
        chunks_a,
        np.diff(section_positions)
    )
]
airfoil_pairs_b:list[WingSegment] = [
    WingSegment(a, b, length=length)
    for (a, b), length
    in zip(
        chunks_b,
        np.diff(section_positions)
    )
]

In [ ]:
chunks_a[1][0].plot()

In [ ]:
fig, axs = plt.subplots(len(chunks_a+chunks_b),2, figsize=(20,15))
for axr, (a,b) in zip(axs,chunks_a+chunks_b):
    a.plot(axr[0])
    b.plot(axr[1])

In [ ]:
import pyvista as pv
pt = pv.Plotter()
offset = 30
for chunk in airfoil_pairs_a:
    offset += chunk.length/2
    m = chunk.to_mesh()
    pt.add_mesh(m.translate((offset,0,0)), opacity=0.5)
    offset += chunk.length/2
offset = -30
for chunk in airfoil_pairs_b:
    offset -= chunk.length/2
    m = chunk.to_mesh()
    pt.add_mesh(m.translate((offset,0,0)), opacity=0.5)
    offset -= chunk.length/2
pt.show()

In [ ]:
from itertools import chain, repeat, count
foam_cut_list = []
for side, wing_segment in chain(
        zip(
            repeat("A"),
            airfoil_pairs_a
        ),
        zip(
            repeat("B"),
            airfoil_pairs_b
        )
    ):
    bounds = wing_segment.to_mesh().bounds
    foam_cut_list.append({
        "side"            : side,
        "foam_width_mm"  : bounds.x_max-bounds.x_min,
        "foam_depth_mm": np.ceil((bounds.y_max-bounds.y_min + 10)/10)*10,
        "foam_height_mm"    : 30 if (bounds.z_max-bounds.z_min) < 30 else 50,
        "wing_segment"    : wing_segment,
    })
segments = pd.DataFrame(foam_cut_list)
segments

In [ ]:
from util.pyvista_mock import axis
from util.path_planning import (
    blur1d,
    map_to_range,
    project_line_to_plane,
    ensure_closed,
    deflection_angle,
)
from airfoil.util import remove_sequential_duplicates, linear_resampling_to_length
from util.pyvista_helpers import create_ruled_surface
from dataclasses import dataclass, replace, field


@dataclass
class MachineSetup:
    wing_segment:WingSegment
    foam_width :float
    foam_depth :float
    foam_height:float
    plane_spacing:float
    decomposer:Decomposer = field(default_factory=lambda:Decomposer())
    max_cut_speed_mm_s:float = 300
    min_cut_speed_mm_s:float = 230
    travel_speed:float       = 1000
    
    def with_recentered_part(self):
        foam_center = np.array([
            0,
            self.foam_depth/2,
            self.foam_height/2,
        ])
        offset = foam_center - self.wing_segment.bounding_center()
        return replace(
            self,
            wing_segment = self.wing_segment.with_translation(offset[-2:])
        )

    def plot(self, state:tuple[float,float,float,float]|ArrayLike):
        _state = np.array(state)
        mesh_foam = pv.Box((
            -self.foam_width/2,self.foam_width/2,
            0,self.foam_depth,
            0,self.foam_height
        ))
        state_a, state_b = self.state_to_line(*_state)
        mesh_state = pv.Line(state_a,state_b)
        decomposer = Decomposer() # TODO: use the class decomposer?
        mesh_target = self.wing_segment.to_mesh(decomposer)

        a,b, speed = self.prepare_cut_surface()

        cut_surface = create_ruled_surface(a,b)
        cut_surface.cell_data["speed mm/s"] = speed[1:]

        instructions = self.instructions()
        ia = np.insert(instructions[:,1:3], 0, -self.plane_spacing/2, axis=-1)
        ib = np.insert(instructions[:,3:5], 0,  self.plane_spacing/2, axis=-1)

        pt = pv.Plotter()
        pt.add_mesh(pv.MultipleLines(a),"green")
        pt.add_mesh(pv.MultipleLines(b),"green")

        pt.add_mesh(pv.MultipleLines(ia),"purple")
        pt.add_mesh(pv.MultipleLines(ib),"purple")

        pt.add_mesh(
            mesh_foam.extract_all_edges(),
            color="teal",
            line_width=2,
        )
        pt.add_mesh(
            cut_surface,
            scalars="speed mm/s",
            cmap="viridis",
            opacity=0.5
        )
        pt.add_mesh(mesh_state, color="red", line_width=2)
        if mesh_target:
            pt.add_mesh(mesh_target)
        pt.add_mesh(axis(
            (-self.plane_spacing/2, *_state[:2]), side="L"
        ), color="white", opacity=0.1)
        pt.add_mesh(axis(
            ( self.plane_spacing/2, *_state[2:]), side="R"
        ), color="white", opacity=0.1)
        pt.camera_position = (
            (-self.foam_width*1.1,-self.foam_width*3,self.foam_height*3),
            (-self.foam_width*0.3,0,0),
            (0,0,1)
        )
        pt.enable_parallel_projection()
        pt.show()


    def state_to_line(self, x:float, y:float, z:float, a:float):
        return (
            (-self.plane_spacing/2,x,y),
            ( self.plane_spacing/2,z,a),
        )

    def states_to_curves(self,states):
        x,y,z,a=states.T
        return (
            np.vstack((np.ones_like(x) * -self.plane_spacing/2, x,y)).T,
            np.vstack((np.ones_like(x) *  self.plane_spacing/2, z,a)).T,
        )
    
    def prepare_cut_surface(self):
        a, b = self.wing_segment.decompose(self.decomposer)
        a = ensure_closed(remove_sequential_duplicates(np.concat(a)))
        b = ensure_closed(remove_sequential_duplicates(np.concat(b)))
        a_3d = np.insert(a, 0, -self.wing_segment.length/2, axis=-1)
        b_3d = np.insert(b, 0,  self.wing_segment.length/2, axis=-1)
        afa_projected = []
        afb_projected = []
        for a_3di, b_3di in zip(
            a_3d,
            b_3d,
        ):
            afa_projected.append(project_line_to_plane(a_3di, b_3di, "yz", -self.plane_spacing/2))
            afb_projected.append(project_line_to_plane(a_3di, b_3di, "yz",  self.plane_spacing/2))
        afa_projected = np.array(afa_projected)
        afb_projected = np.array(afb_projected)
        assert all(np.linalg.norm(a,axis=-1)!=0)
        assert all(np.linalg.norm(b,axis=-1)!=0)
        speed = map_to_range(
            blur1d(
                np.max([
                    deflection_angle(a), # error on NAN
                    deflection_angle(b),
                ],axis=0),
                count=31,
                std=6
            ),
            self.max_cut_speed_mm_s,
            self.min_cut_speed_mm_s
        )
        speed_multiplier = (
             (np.linalg.norm(np.diff(afa_projected))/np.linalg.norm(np.diff(a)))
            +(np.linalg.norm(np.diff(afb_projected))/np.linalg.norm(np.diff(b)))
        )/2
        return afa_projected, afb_projected, (speed*speed_multiplier)

    def instructions(self, record_name:str|None=None):
        a,b,speed = self.prepare_cut_surface()
        ab_all = np.concat([a,b], axis=0)
        max_y_lead_in_out = np.max(ab_all, axis=0)[1]
        li = np.array([
            [                 0,                  0 ],
            [                -5,                  0 ],
            [                -5, self.foam_height+20 ],
            [ self.foam_depth+5, self.foam_height+20 ],
            [ max(self.foam_depth+5, max_y_lead_in_out+5), self.foam_height/2 ],
        ])
        lo = np.array([
            [ self.foam_depth+5, self.foam_height/2],
        ])
        # li = linear_resampling_to_length(li,3)
        # lo = linear_resampling_to_length(lo,3)
        instrucitons = np.concat([
            np.concat(
                [
                    np.full((len(li),1), self.travel_speed),
                    li,
                    li
                ],
                axis=-1
            ),
            np.concat([speed.reshape(-1,1), a[:,1:], b[:,1:]], axis=-1),
            np.concat(
                [
                    np.full((len(lo),1), self.travel_speed),
                    lo,
                    lo
                ],
                axis=-1
            ),
        ])

        if record_name is not None:
            rec = (
                f'''{{\n'''
                f'''    "self": {self}\n'''
                f'''    "left": {self.wing_segment.left.points}\n'''
                f'''    "right": {self.wing_segment.right.points}\n'''
                f'''}}\n'''
            )
            from pathlib import Path
            
            folder = Path("./records/")
            folder.mkdir(exist_ok=True)
            file = folder / f"{pd.Timestamp.now():%Y-%m-%d %H%M} {record_name}.txt"
            file.write_text(rec)

        return instrucitons


In [ ]:
# seg = 0 # done
# seg = 1 # done
# seg = 2 # done
# seg = 3 # done
# seg = 4 # done
# seg = 5 # done
# seg = 6 # done
# seg = 7 # done
# seg = 8 # done

seg = 9

ms = MachineSetup(
    wing_segment = segments.loc[seg, "wing_segment"  ],
    foam_width   = segments.loc[seg, "foam_width_mm" ],
    foam_depth   = segments.loc[seg, "foam_depth_mm" ],
    foam_height  = segments.loc[seg, "foam_height_mm"],
    plane_spacing= 227,
    decomposer=Decomposer(buffer=0.5, segment_target_length=1),
    max_cut_speed_mm_s=240,
    min_cut_speed_mm_s=130,
    travel_speed=1000
).with_recentered_part()

print(
    f'please configure machine as shown:\n   Foam '
    f'{segments.loc[seg, "foam_width_mm" ]:.0f} x '
    f'{segments.loc[seg, "foam_depth_mm" ]:.0f} x '
    f'{segments.loc[seg, "foam_height_mm"]:.0f} mm'
    f'\n   Plane spacing: {ms.plane_spacing:.0f} mm'
)
state = np.array([0,0,0,0])
ms.plot(state)
None

In [ ]:
from util.serial import CNC
cnc = CNC()

In [ ]:
cnc.home()

In [ ]:
cnc.status()

In [ ]:
cnc.metric()
cnc.set_position(0,0,0,0)
cnc.absolute()

In [ ]:
# initial square-up dont change for now
cnc.travel(
    x=110-50,y=35,
    z=99 -50,a=30.5
)

In [ ]:
# initial square-up dont change for now
cnc.travel(
    x=0,y=-0.5,
    z=0,a=-0.5,
)

In [ ]:
cnc.set_position(0,0,0,0)
cnc.absolute()

In [ ]:
cnc.travel(
    x=0,y=0,
    z=0,a=0,
)

In [ ]:
cnc.send_g1_commands(ms.instructions(record_name=f"Segment {seg}"))

In [ ]:
cnc.travel(
    x=0,y=5,
    z=0,a=5,
)
cnc.travel(
    x=0,y=0,
    z=0,a=0,
)